<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow2/Customise/DataPipeline/TFDataset_pandas_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Datasets from different sources

In this reading notebook, we will explore a few of the ways in which we can load data into a `tf.data.Dataset` object.

In [1]:
%%bash
pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU numpy pandas matplotlib seaborn scikit-learn
pip install --no-cache-dir -qU tensorflow
pip check

No broken requirements found.


In [2]:
import os
os.environ['TP_CPP_MIN_LOG_LEVEL'] = '2'

import warnings
warnings.filterwarnings('ignore')

import numpy as np
np.random.seed(42)
import pandas as pd

import tensorflow as tf
tf.keras.utils.set_random_seed(42)
tf.get_logger().setLevel('ERROR')
print(tf.__version__)

2.11.0


## The `from_tensor_slices` and `from_tensors` methods

We will start by looking at the `from_tensor_slices` and the `from_tensors` methods.

Both static methods are used to create datasets from Tensors or Tensor-like objects, such as numpy arrays or python lists. We can also pass in tuples and dicts of arrays or lists. The main distinction between the `from_tensor_slices` function and the `from_tensors` function is that the `from_tensor_slices` method will interpret the first dimension of the input data as the number of elements in the dataset, whereas the `from_tensors` method always results in a Dataset with a single element, containing the Tensor or tuple of Tensors passed.

In [3]:
# Create a random tensor with shape (3, 2)

example_tensor = tf.random.uniform(shape=[3, 2])
print(example_tensor.shape)

(3, 2)


In [4]:
# Create two Datasets, using each static method
# Print the element_spec for each

dataset1 = tf.data.Dataset.from_tensor_slices(
    tensors=example_tensor)
print(dataset1.element_spec)
for elem in dataset1:
    print(elem)

dataset2 = tf.data.Dataset.from_tensors(
    tensors=example_tensor)
print(dataset2.element_spec)
for elem in dataset2:
    print(elem)

TensorSpec(shape=(2,), dtype=tf.float32, name=None)
tf.Tensor([0.68789124 0.48447883], shape=(2,), dtype=float32)
tf.Tensor([0.9309944 0.252187 ], shape=(2,), dtype=float32)
tf.Tensor([0.73115396 0.89256823], shape=(2,), dtype=float32)
TensorSpec(shape=(3, 2), dtype=tf.float32, name=None)
tf.Tensor(
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]
 [0.73115396 0.89256823]], shape=(3, 2), dtype=float32)


As seen above, creating the Dataset using the `from_tensor_slices` method slices the given array or Tensor along the first dimension to produce a set of elements for the Dataset.

This means that although we could pass any Tensor - or tuple of Tensors - to the `from_tensors` method, the same cannot be said of the `from_tensor_slices` method, which has the additional requirement that each Tensor in the list has the same size in the zeroth dimension.

In [5]:
# Create three Tensors with different shapes

tensor1 = tf.random.uniform(shape=[10, 2, 2])
tensor2 = tf.random.uniform(shape=[10, 1])
tensor3 = tf.random.uniform(shape=[9, 2, 2])

We cannot create a Dataset using the `from_tensor_slices` method from a list of `tensor1` and `tensor3` since they do not have the same size in the first dimension:

In [6]:
# Try to create a Dataset from tensor1 and tensor3 using from_tensor_slices - this will raise an error

dataset = tf.data.Dataset.from_tensor_slices(
    tensors=(tensor1, tensor3))

ValueError: Dimensions 10 and 9 are not compatible

However, we can of course create a Dataset from this tuple using the `from_tensors` method, which interprets the tuple as a single element.

In [9]:
# Create a Dataset from tensor1 and tensor3 using from_tensors

dataset = tf.data.Dataset.from_tensors(
    tensors=(tensor1, tensor3))
print(dataset.element_spec)

(TensorSpec(shape=(10, 2, 2), dtype=tf.float32, name=None), TensorSpec(shape=(9, 2, 2), dtype=tf.float32, name=None))


Although `tensor1` and `tensor2` do not have the same shape, or even same rank (number of dimensions), we can still use the `from_tensor_slices` method to form a dataset from a list of these tensors, since they have the same size in the first dimension.

In [10]:
# Create a Dataset from tensor1 and tensor2

dataset = tf.data.Dataset.from_tensor_slices(
    tensors=(tensor1, tensor2))
print(dataset.element_spec)

(TensorSpec(shape=(2, 2), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))


In the above, the first dimension was interpreted as the number of elements in the Dataset, as expected.

### Creating Datasets from numpy arrays

We can also use the `from_tensor_slices` and `from_tensors` methods to create Datasets from numpy arrays. In fact, behind the scenes, the numpy array is converted to a set of `tf.constant` operations to populate the Tensor in the TensorFlow graph.

In [11]:
# Create a numpy array dataset

numpy_array = np.array([
    [[1, 2], [3, 4]],
    [[5, 6], [7, 8]],
    [[9, 10], [11, 12]]
])
print(numpy_array.shape)

(3, 2, 2)


In [12]:
# Create two Datasets, using each static method

dataset1 = tf.data.Dataset.from_tensor_slices(
    tensors=numpy_array)
print(dataset1.element_spec)

dataset2 = tf.data.Dataset.from_tensors(
    tensors=numpy_array)
print(dataset2.element_spec)

TensorSpec(shape=(2, 2), dtype=tf.int64, name=None)
TensorSpec(shape=(3, 2, 2), dtype=tf.int64, name=None)


As before, `from_tensors` interprets the entire array as a single element, whereas `from_tensor_slices` slices the array along the first dimension to form the elements.

### Creating Datasets from pandas DataFrames
A pandas DataFrame can be easily converted to a Dataset using the `from_tensor_slices` method. 
#### The Balloons dataset
A pandas DataFrame can be loaded from a CSV file. We will use the [Balloons dataset](https://archive.ics.uci.edu/ml/datasets/Balloons) to demonstrate. This dataset is stored in a CSV file, and contains a list of attributes describing instances of a balloon inflation experiment, such as the colour and size of the balloon, the age of the person who performed the attempted inflation, and the way in which they did it. Finally, there is the target column "Inflated", which is either `T` for True, or `F` for False, indicating whether or not the person managed to inflate the balloon.

In [13]:
os.makedirs(name="./data", exist_ok=True)

In [15]:
# Load the CSV file into a Dataframe

pandas_dataframe = pd.read_csv("./data/balloon_dataset.csv")

In [16]:
# Inspect the data

pandas_dataframe.head()

,Colour,Size,Act,Age,Inflated
0,YELLOW,SMALL,STRETCH,ADULT,T
1,YELLOW,SMALL,STRETCH,ADULT,T
2,YELLOW,SMALL,STRETCH,CHILD,F
3,YELLOW,SMALL,DIP,ADULT,F
4,YELLOW,SMALL,DIP,CHILD,F


To convert the DataFrame to a Dataset, we first convert the DataFrame to a dictionary. By doing this, we preserve the column names as the dictionary labels.

**Note**: A Dataset can be formed from either a tuple or a dict of Tensors. We saw above a number of Datasets being formed from a tuple. The only distinction for a Dataset formed from a dict is that the Dataset items will be dicts accessed by key, rather than tuples accessed by index.

In [17]:
# Convert the DataFrame to a dict

dataframe_dict = dict(pandas_dataframe)
print(dataframe_dict.keys())

dict_keys(['Colour', 'Size', 'Act', 'Age', 'Inflated'])


We can now run the `from_tensor_slices` method on this `dict` and print the resulting Dataset `element_spec`, as well as an example element. Note that since we formed the Dataset from a `dict`, we see the column (dictionary) names in the `element_spec`.

In [18]:
# Create the Dataset

pandas_dataset = tf.data.Dataset.from_tensor_slices(
    tensors=dataframe_dict)

In [19]:
# View the Dataset element_spec

pandas_dataset.element_spec

{'Colour': TensorSpec(shape=(), dtype=tf.string, name=None),
 'Size': TensorSpec(shape=(), dtype=tf.string, name=None),
 'Act': TensorSpec(shape=(), dtype=tf.string, name=None),
 'Age': TensorSpec(shape=(), dtype=tf.string, name=None),
 'Inflated': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [20]:
# Iterate the Dataset

next(iter(pandas_dataset))

{'Colour': <tf.Tensor: shape=(), dtype=string, numpy=b'YELLOW'>,
 'Size': <tf.Tensor: shape=(), dtype=string, numpy=b'SMALL'>,
 'Act': <tf.Tensor: shape=(), dtype=string, numpy=b'STRETCH'>,
 'Age': <tf.Tensor: shape=(), dtype=string, numpy=b'ADULT'>,
 'Inflated': <tf.Tensor: shape=(), dtype=string, numpy=b'T'>}

## Creating Datasets directly from CSV Files

The TensorFlow experimental library contains a variety of functions and classes contributed by the community that may not be ready for release into the main TensorFlow library in their immediate form, but which may be included in TensorFlow in the future. One such useful experimental function is the `tf.data.experimental.make_csv_dataset` function. This allows us to read CSV data from the disk directly into a Dataset object.

We will run the function on the example CSV file from disk, and specify the batch size and the name of the target column, which is used to structure the Dataset into an `(input, target)` tuple.

**Note:** Because of the ephemeral nature of the `experimental` package, you may well get warnings printed in the console when using a function or class contained in the package for the first time.

In [21]:
# Create the Dataset from the CSV file

csv_dataset = tf.data.experimental.make_csv_dataset(
    file_pattern='./data/balloon_dataset.csv',
    batch_size=1, label_name='Inflated')

To check that we've loaded our Dataset correctly, let's print the `element_spec`:

In [22]:
# View the Dataset element_spec

csv_dataset.element_spec

(OrderedDict([('Colour', TensorSpec(shape=(1,), dtype=tf.string, name=None)),
              ('Size', TensorSpec(shape=(1,), dtype=tf.string, name=None)),
              ('Act', TensorSpec(shape=(1,), dtype=tf.string, name=None)),
              ('Age', TensorSpec(shape=(1,), dtype=tf.string, name=None))]),
 TensorSpec(shape=(1,), dtype=tf.string, name=None))

In [23]:
# Iterate the Dataset

next(iter(csv_dataset))

(OrderedDict([('Colour',
               <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'YELLOW'], dtype=object)>),
              ('Size',
               <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'LARGE'], dtype=object)>),
              ('Act',
               <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'DIP'], dtype=object)>),
              ('Age',
               <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'CHILD'], dtype=object)>)]),
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'F'], dtype=object)>)

Note that in the above Dataset, the target column `Inflated` does not have a key, since it is uniquely accessible as the second element of the tuple, whereas the attributes which reside as a dictionary of Tensors in the first element retain their labels so we can distinguish them.

## Further reading and resources

* https://www.tensorflow.org/guide/data
* https://www.tensorflow.org/tutorials/load_data/csv
* https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
* https://www.tensorflow.org/api_docs/python/tf/data/Dataset